## About this notebook

This notebook segments files selected for calculating alignment in each round.

Input:
- pathway to a directory containing selected files for the segmentation (im_to_segment)
- list of wells to process

Output:
- segmented masks of selected channels

## Fill in info about the experiment to process

In [1]:
# define a pathway to a directory that contains images for segmentation (im_to_segment)
path_data = r'C:\Users\gases\Desktop\DataForAyra\output_images'

# list of wells to be processed (usually names as 'A3')
well_list = ['A1']
# pathway to a directory to save segmented images
path_save = r'C:\Users\gases\Desktop\DataForAyra\segmented'

# expected diameter of objects for segmentation in pixels (default 30)
selected_diameter = 30

## Prepare for segmentation

In [2]:
import os
from tifffile import imread
import pickle
from skimage.io import imsave
import tensorflow as tf
import cellpose
from cellpose import models
from cellpose import utils

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
#Is cellpose using the GPU, reports True or False
cellpose.core.use_gpu(gpu_number=0, use_torch=True)

True

In [5]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:36:24_Pacific_Standard_Time_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0
Mon Aug 26 17:22:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 538.59                 Driver Version: 538.59       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro P620                  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P0    

In [6]:
try:
    #os.mkdir(save_dir)
    os.mkdir(path_save)
    print('Directory for saving created.')
except:
    print('Directory not created.')

Directory for saving created.


In [7]:
# load cellpose model
#model = models.Cellpose(gpu=True, model_type='cyto')
model = models.Cellpose(gpu=False, model_type='cyto')

## Process selected wells

In [8]:
# loop for segmentation 

for myWell in well_list:
    
    file_list = os.listdir(os.path.join(path_data,myWell))
    
    try:
        os.mkdir(os.path.join(path_save,myWell))
        print(f'Directory for saving {myWell} labels created.')
    except:
        pass

    for file_name in file_list: 

        # specify pathway to the image
        im_path = os.path.join(path_data,myWell,file_name)

        # get an image
        im = imread(im_path)

        # segment the right plane
        labels, _, _, _ = model.eval(im, diameter=selected_diameter, channels=[0,0])

        # save segmentation
        path_save_temp = os.path.join(path_save,myWell,file_name.replace('im2segment','labels'))
        imsave(path_save_temp,labels)         

Directory for saving A1 labels created.


more than 65535 masks in image, masks returned as np.uint32
c:\Users\gases\.conda\envs\GPU_Segment_4i\lib\site-packages\ipykernel_launcher.py:26: UserWarning: C:\Users\gases\Desktop\DataForAyra\segmented\A1\labels_A1_round_000.tif is a low contrast image


KeyboardInterrupt: 